In [ ]:
import gzip
import math
import pickle

import seaborn as sns

import matplotlib.pyplot as plt
from rdkit.Chem import PandasTools, Crippen


import warnings
import numpy as np 
import pandas as pd
from rdkit import Chem
from rdkit import rdBase
from rdkit import DataStructs
rdBase.DisableLog('rdApp.error')
warnings.filterwarnings("ignore")

import argparse
from rdkit.Chem import PandasTools
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
import os
from mol_metrics import *
import logging
from rdkit.Chem import PandasTools, QED, Descriptors, rdMolDescriptors
import re


In [ ]:
# ===========================
# Chemical properties calculation for training dataset:
def evaluation_training(training_smiles):
    
    train_smiles = training_smiles

    training_length = len(train_smiles)
    # chemical properties based on novel_smiles_unique
    # Convert SMILES to Mol objects and filter out invalid entries
    training_data = pd.Series(train_smiles).apply(Chem.MolFromSmiles).dropna()
    training_data = training_data[training_data.apply(lambda x: x.GetNumAtoms() > 1)]
    
   
    
    if not training_data.empty:
        # Compute QED, solubility, and SA scores
        training_qed = training_data.apply(QED.qed)
        training_solubility = training_data.apply(solubility)
        training_sa = training_data.apply(SA)

        # Compute means
        training_mean_qed = training_qed.mean() if not training_qed.empty else 0
        training_mean_solubility = training_solubility.mean() if not training_solubility.empty else 0
        training_mean_sa = training_sa.mean() if not training_sa.empty else 0
    else: 
        training_mean_qed = 0
        training_mean_solubility = 0
        training_mean_sa = 0





    return  training_length, training_qed, training_solubility, training_sa, training_mean_qed, training_mean_solubility, training_mean_sa



# Chemical properties calculation for generated dataset:
def evaluation_generated(generated_smiles, training_smiles):


    """
    args:
        - generated_smiles: a list of generated SMILES strings
        - training_smiles: training SMILES dataset

    returns:
        - validity: ratio of valid SMILES strings in generated SMILES stirngs
        - uniqueness: ratio of unique SMILES strings in the valid SMILES strings
        - novelty: ratio of novel SMILES strings in the unique SMILES strings
        - diversity: diversity score of the novel SMILES strings
    """

    # SMILES to molecules
    generated_mols = np.array([Chem.MolFromSmiles(s) for s in generated_smiles if len(s.strip())])
    
    if len(generated_mols) == 0:
        print('No SMILES data is generated, please pre-train the generator again!')
        return

    else:
        train_smiles = training_smiles

        valid_smiles = [Chem.MolToSmiles(mol) for mol in generated_mols if mol != None and mol.GetNumAtoms() > 1 and Chem.MolToSmiles(mol) != ' ']
        unique_smiles = list(set(valid_smiles))
        novel_smiles_unique = [smi for smi in unique_smiles if smi not in train_smiles]
        novel_length = len(novel_smiles_unique)
    

        # chemical properties based on novel_smiles_unique
        # Convert SMILES to Mol objects and filter out invalid entries
        novel_data = pd.Series(novel_smiles_unique).apply(Chem.MolFromSmiles).dropna()
        novel_data = novel_data[novel_data.apply(lambda x: x.GetNumAtoms() > 1)]

        
        if not novel_data.empty:
            # Compute QED, solubility, and SA scores
            novel_qed = novel_data.apply(QED.qed)
            novel_solubility = novel_data.apply(solubility)
            novel_sa = novel_data.apply(SA)

            # Compute means
            novel_mean_qed = novel_qed.mean() if not novel_qed.empty else 0
            novel_mean_solubility = novel_solubility.mean() if not novel_solubility.empty else 0
            novel_mean_sa = novel_sa.mean() if not novel_sa.empty else 0
        else: 
            novel_mean_qed = 0
            novel_mean_solubility = 0
            novel_mean_sa = 0





    return  novel_length, novel_qed, novel_solubility, novel_sa, novel_mean_qed, novel_mean_solubility, novel_mean_sa




In [ ]:
def draw_distributions2(
    training_smiles, 
    generated_smiles,  
    training_qed, 
    training_solubility, 
    training_sa, 
    training_mean_qed, 
    training_mean_solubility, 
    training_mean_sa,
    save_to_file = 'Distributions',
    optimized_property = None,
    property = None
):
   
    if property is not None:
        
        novel_length, novel_qed, novel_solubility, novel_sa, novel_mean_qed, novel_mean_solubility, novel_mean_sa = evaluation_generated(generated_smiles, training_smiles)
        
        score_map = {
            "qed": (training_qed, novel_qed),
            "solubility": (training_solubility, novel_solubility),
            "sa": (training_sa, novel_sa)
        }

        mean_score_map = {
            "qed": (training_mean_qed, novel_mean_qed),
            "solubility": (training_mean_solubility, novel_mean_solubility),
            "sa": (training_mean_sa, novel_mean_sa)
        }

        if property in score_map:
            training_scores_df, novel_scores_df = score_map[property]
            training_mean_score, novel_mean_score = mean_score_map[property]
        else:
            raise ValueError(f"Invalid property: {property}")


    
        # Mean scores
        print('Mean REAL {}: {:.2f}'.format(property, training_mean_score))
        print('Mean GAN NOVEL {}: {:.2f}'.format(property, novel_mean_score))

        
        selected_training_scores_df = training_scores_df.sample(n=novel_length, replace=False, random_state=0)

        print(len(selected_training_scores_df))
        print(novel_length)
        print(len(novel_scores_df))

        
        # X-axis
        if property == 'qed':
            xlabel = 'Drug-likeness (QED) score'
        elif property == 'solubility':
            xlabel = 'Solubility (logP) score'
        elif property == 'sa':
            xlabel = 'Synthesizability (SA) score'
            
        # Plot distribution figures for real dataset and GAN
        plt.subplots(figsize=(12,7))

        bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
        plt.xticks(np.arange(0.05, 1.05, 0.1), fontsize=15) 
        plt.xlabel(xlabel, size = 15)
        plt.yticks(fontsize = 15)
        plt.ylabel('Number of molecules', size = 15)
        
        plt.hist([selected_training_scores_df, novel_scores_df], bins=bins, alpha=0.9, label=['Real molecules', 'Generated molecules'], color=['royalblue', 'lightgreen'])    
        

        plt.legend(loc='upper left', prop={'size': 15})
        
        
        # Check if the folder exists
        if not os.path.exists(save_to_file):
            os.makedirs(save_to_file)

        plt.savefig(f"{save_to_file}/{optimized_property}_{property}.pdf")

In [ ]:
data = []
with open('zinc250k.csv', "r") as f:
    for line in f.readlines()[1:]:
        data.append(line.split(",")[1])

# Training canonical SMILES
training_smiles = [Chem.MolToSmiles(Chem.MolFromSmiles(sm)) for sm in data]
training_smiles = list(set(training_smiles))
training_length, training_qed, training_solubility, training_sa, training_mean_qed, training_mean_solubility, training_mean_sa = evaluation_training(training_smiles)


In [ ]:
print(training_length)

generated_smiles_QED_qed = []
file_path = "QED/Results/QED/ew=0.022_gw=1e-05/Alphas/0.5/logs/best_QED_generated_data.csv"

with open(file_path, "r") as f:
    for line in f.readlines()[1:]:
        generated_smiles_QED_qed.append(line.strip())




In [ ]:
draw_distributions2(
    training_smiles, 
    generated_smiles_QED_qed,  
    training_qed, 
    training_solubility, 
    training_sa, 
    training_mean_qed, 
    training_mean_solubility, 
    training_mean_sa,
    save_to_file = 'Distributions',
    optimized_property = 'single',
    property = 'qed'
)

In [ ]:
print(training_length)

generated_smiles_Solubility_solubility = []
file_path = "Solubility/Results/Solubility/ew=0.022_gw=1e-05/Alphas/0.5/logs/best_Solubility_generated_data.csv"

with open(file_path, "r") as f:
    for line in f.readlines()[1:]:
        generated_smiles_Solubility_solubility.append(line.strip())





In [ ]:
draw_distributions2(
    training_smiles, 
    generated_smiles_Solubility_solubility,  
    training_qed, 
    training_solubility, 
    training_sa, 
    training_mean_qed, 
    training_mean_solubility, 
    training_mean_sa,
    save_to_file = 'Distributions',
    optimized_property = 'single',
    property = 'solubility'
)

In [ ]:
print(training_length)

generated_smiles_SA_sa = []
file_path = "SA/Results/SA/ew=0.022_gw=1e-05/Alphas/0.5/logs/best_SA_generated_data.csv"

with open(file_path, "r") as f:
    for line in f.readlines()[1:]:
        generated_smiles_SA_sa.append(line.strip())




In [ ]:
draw_distributions2(
    training_smiles, 
    generated_smiles_SA_sa,  
    training_qed, 
    training_solubility, 
    training_sa, 
    training_mean_qed, 
    training_mean_solubility, 
    training_mean_sa,
    save_to_file = 'Distributions',
    optimized_property = 'single',
    property = 'sa'
)

In [ ]:
print(training_length)

generated_smiles_ALL = []
file_path = "ALL/Results/ALL/ew=0.022_gw=1e-05/Alphas/0.5/logs/best_ALL_generated_data.csv"

with open(file_path, "r") as f:
    for line in f.readlines()[1:]:
        generated_smiles_ALL.append(line.strip())





In [ ]:
draw_distributions2(
    training_smiles, 
    generated_smiles_ALL,  
    training_qed, 
    training_solubility, 
    training_sa, 
    training_mean_qed, 
    training_mean_solubility, 
    training_mean_sa,
    save_to_file = 'Distributions',
    optimized_property = 'multi',
    property = 'qed'
)

In [ ]:
draw_distributions2(
    training_smiles, 
    generated_smiles_ALL,  
    training_qed, 
    training_solubility, 
    training_sa, 
    training_mean_qed, 
    training_mean_solubility, 
    training_mean_sa,
    save_to_file = 'Distributions',
    optimized_property = 'multi',
    property = 'solubility'
)

In [ ]:
draw_distributions2(
    training_smiles, 
    generated_smiles_ALL,  
    training_qed, 
    training_solubility, 
    training_sa, 
    training_mean_qed, 
    training_mean_solubility, 
    training_mean_sa,
    save_to_file = 'Distributions',
    optimized_property = 'multi',
    property = 'sa'
)